In [1]:
import random
import time

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import torch

from src.model.activation import Relu, LeakyRelu, Linear, Sigmoid
from src.model.base import NeuralNetwork
from src.model.layer import Convolutional, Input3D, Input, FullyConnected, Flatten
from src.model.loss import CategoricalCrossEntropy
from src.model.metric import AccuracyOneHot
from src.model.optimizer import Adam, SGD

In [2]:
# train_data = [
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }
    
# ]

In [3]:
# layers = [
#     Convolutional(LeakyRelu(), filters_num=2, kernel_size=2),
#     Convolutional(LeakyRelu(), filters_num=2, kernel_size=2),
#     Flatten(),
#     #FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input3D((1, 5, 5)),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )


# start_time = time.time()
# epochs = 10
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_111')
# import sys
# sys.exit(0)

In [4]:
def plot_digit(image, digit):
    image = torch.tensor(image).numpy().reshape(28, 28)
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.title(f"Predicted digit: {digit}")
    plt.axis("off")
    plt.show()

    return

In [5]:
def get_digit(onehot_array):
    for index, value in enumerate(onehot_array):
        if value == 1.0:
            return index
    return None

In [6]:
def get_onehot_digit(digit):
    output = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    return output[-digit:] + output[:-digit]

In [7]:
# def get_digits_data():

#     train_dataset = []

#     train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

#     for index in train.index:
#         input_values = [
#             float(val) / 255.0 for val in train.loc[index].drop("label").values
#         ]
#         train_dataset.append(
#             {
#                 "input": input_values,
#                 "output": get_onehot_digit(int(train.loc[index]["label"])),
#             }
#         )

#     random.seed(1)
#     random.shuffle(train_dataset)
#     return train_dataset[:800], train_dataset[800:1000]

# train_dataset, test_dataset = get_digits_data()

# layers = [
#     FullyConnected(256, LeakyRelu()),
#     FullyConnected(128, LeakyRelu()),
#     FullyConnected(64, LeakyRelu()),
#     FullyConnected(10, Linear()),
# ]
# nn = NeuralNetwork(
#     Input(784),
#     layers,
#     optimizer=Adam(learning_rate=0.001),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction="argmax",
#     on_cuda=False
# )

# start_time = time.time()

# epochs = 15
# history = nn.fit(train_dataset, test_dataset, epochs=epochs, batch_size=1, verbose=True)

# print("--- %s seconds ---" % (time.time() - start_time))

# prediction = nn.predict(test_dataset)

# for index, predicted_digit in enumerate(prediction[:10]):
#     plot_digit(test_dataset[index]['input'], get_digit(predicted_digit))

In [8]:
def plot_digit_cnn(image, digit):
    image = torch.tensor(image).numpy()
    plt.figure()
    plt.title(f"Predicted digit: {digit}")
    plt.imshow(image, cmap="gray")
    plt.axis("off")
    plt.show()

    return

In [9]:
def get_digits_data_cnn():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": [torch.tensor(input_values).reshape(28, 28).tolist()],
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    # print(train_dataset[:2])
    # import sys
    # sys.exit(1)
    
    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:800], train_dataset[800:1000]

In [10]:
train_data, test_data = get_digits_data_cnn()

layers = [
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=4),
    Convolutional(LeakyRelu(), filters_num=8, kernel_size=3),
    Convolutional(LeakyRelu(), filters_num=16, kernel_size=2),
    Flatten(),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(10, Linear())
]
nn = NeuralNetwork(
    Input3D((1, 28, 28)),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction='argmax',
    on_cuda=False
)

#plot_digit(train_data[0]['input'][0])

start_time = time.time()
epochs = 15
nn.fit(train_data, test_data, epochs=epochs, batch_size=1, verbose=True)
print('exec time: ', time.time() - start_time)

prediction = nn.predict(test_data)

for index, predicted_digit in enumerate(prediction[:10]):
    plot_digit_cnn(test_data[index]['input'][0], get_digit(predicted_digit))

Epoch: 1/15, train loss: 1.9087, train Accuracy: 0.7275, test loss: 1.7215, test Accuracy: 0.855, epoch time: 50.854s


KeyboardInterrupt: 

In [ ]:
# train_data = [
#     {
#         'input': [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# layers = [
#     FullyConnected(25, LeakyRelu()),
#     FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input(25),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )

# start_time = time.time()
# epochs = 200
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_222')